In [2]:
import os
import polars as pl
from matplotlib import pyplot as plt
import numpy as np
import torch
from ase.data import chemical_symbols
from ase.atoms import Atoms
from ase.visualize import view
from fairchem.core.datasets import LmdbDataset
from fairchem.core.common.relaxation.ase_utils import batch_to_atoms
from torch_geometric.nn.models.schnet import SchNet

In [3]:
state_dict = torch.load('/Users/averyhill/Github/BindingModelMetrics/models/schnet_all_large.pt', map_location='cpu')
state_dict['config']

/var/folders/yf/g4qx_p5929jcvzyg7pwr1rww0000gn/T/ipykernel_31615/1188474524.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/Users/averyhill/Git

{'task': {'dataset': 'trajectory_lmdb',
  'description': 'Regressing to energies and forces for DFT trajectories from OCP',
  'type': 'regression',
  'metric': 'mae',
  'labels': ['potential energy'],
  'grad_input': 'atomic forces',
  'train_on_free_atoms': True,
  'eval_on_free_atoms': True},
 'model': 'schnet',
 'model_attributes': {'hidden_channels': 1024,
  'num_filters': 256,
  'num_interactions': 5,
  'num_gaussians': 200,
  'cutoff': 6.0,
  'use_pbc': True},
 'optim': {'batch_size': 20,
  'eval_batch_size': 8,
  'eval_every': -10000,
  'num_workers': 64,
  'lr_initial': 0.0001,
  'lr_gamma': 0.1,
  'lr_milestones': [313905, 523176, 732446],
  'warmup_factor': 0.2,
  'max_epochs': 15,
  'force_coefficient': 30,
  'warmup_steps': 209270},
 'logger': 'tensorboard',
 'amp': False,
 'cmd': {'identifier': 'schnet_all_large',
  'print_every': 10,
  'seed': 1,
  'timestamp': '2020-10-09-06-00-35-schnet_all_large',
  'checkpoint_dir': './checkpoints/2020-10-09-06-00-35-schnet_all_large'